# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



In [49]:
non_empty_sentences = [sentence for scene in scenes for sentence in scene.split('\n') if sentence]
print('Number of non-empty lines: {}'.format(len(non_empty_sentences)))
non_empty_sentences[:10]

Number of non-empty lines: 3996


["Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.",
 'Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.',
 "Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?",
 "Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.",
 "Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.",
 'Homer_Simpson: I got my problems, Moe. Give me another one.',
 'Moe_Szyslak: Homer, hey, you should not drink to forget your problems.',
 'Barney_Gumble: Yeah, you should only drink to enhance your social skills.',
 "Moe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.",
 "Barney_Gumble: If I ever vote, it'll be for him. (BELCH)"]

In [4]:
scenes[0]

"\nMoe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.\nMoe_Szyslak: What's the matter Homer? You're not your normal effervescent self.\nHomer_Simpson: I got my problems, Moe. Give me another one.\nMoe_Szyslak: Homer, hey, you should not drink to forget your problems.\nBarney_Gumble: Yeah, you should only drink to enhance your social skills."

In [5]:
word_count_non_empty_sentence = [len(sentence.split()) for sentence in non_empty_sentences]
print('Average number of words in non-empty lines: {}'.format(np.average(word_count_non_empty_sentence)))

Average number of words in non-empty lines: 12.255755755755755


## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [6]:
words = text.split()

In [7]:
words[:100]

['Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 "Moe's",
 'Tavern.',
 'Where',
 'the',
 'elite',
 'meet',
 'to',
 'drink.',
 'Bart_Simpson:',
 'Eh,',
 'yeah,',
 'hello,',
 'is',
 'Mike',
 'there?',
 'Last',
 'name,',
 'Rotch.',
 'Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 'Hold',
 'on,',
 "I'll",
 'check.',
 '(TO',
 'BARFLIES)',
 'Mike',
 'Rotch.',
 'Mike',
 'Rotch.',
 'Hey,',
 'has',
 'anybody',
 'seen',
 'Mike',
 'Rotch,',
 'lately?',
 'Moe_Szyslak:',
 '(INTO',
 'PHONE)',
 'Listen',
 'you',
 'little',
 'puke.',
 'One',
 'of',
 'these',
 'days',
 "I'm",
 'gonna',
 'catch',
 'you,',
 'and',
 "I'm",
 'gonna',
 'carve',
 'my',
 'name',
 'on',
 'your',
 'back',
 'with',
 'an',
 'ice',
 'pick.',
 'Moe_Szyslak:',
 "What's",
 'the',
 'matter',
 'Homer?',
 "You're",
 'not',
 'your',
 'normal',
 'effervescent',
 'self.',
 'Homer_Simpson:',
 'I',
 'got',
 'my',
 'problems,',
 'Moe.',
 'Give',
 'me',
 'another',
 'one.',
 'Moe_Szyslak:',
 'Homer,',
 'hey,',
 'you',
 'should',
 'not',
 'drink',
 'to',
 'forget'

In [48]:
from collections import Counter
words = text.split()
counts = Counter(words)
sorted(list(counts), key=counts.get, reverse=True)[:20]


['I',
 'Moe_Szyslak:',
 'the',
 'Homer_Simpson:',
 'a',
 'you',
 'to',
 'of',
 'and',
 'my',
 'in',
 "I'm",
 'that',
 'is',
 'for',
 'your',
 'this',
 'it',
 'Lenny_Leonard:',
 'You']

In [9]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    vocab = sorted(counts, key=counts.get, reverse=True)
    vocab_to_int = {word:idx for idx, word in enumerate(vocab)}
    int_to_vocab = {idx:word for idx, word in enumerate(vocab)}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [10]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokenize_lookup = {'.': '||Period||',
                       ',': '||Comma||',
                       '"': '||Quotation_Mark||',
                       ';': '||Semicolon||',
                       '!': '||Exclamation_Mark||',
                       '?': '||Question_Mark||',
                       '(': '||Left_Parentheses||',
                       ')': '||Right_Parentheses||',
                       '--': '||Dash||',
                       '\n': '||Return||'
                        }
    return tokenize_lookup

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearningRate)`

In [14]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    Input = tf.placeholder(tf.int32, (None, None), name='input')
    Targets = tf.placeholder(tf.int32, (None, None), name='targets')
    LearningRate = tf.placeholder(tf.float32, name='learning_rate')
    return Input, Targets, LearningRate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [15]:
def get_init_cell(batch_size, rnn_size, rnn_layers=1):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #drop = tf.contrib.rnn.core_rnn_cell.DropoutWrapper(lstm, output_keep_prob=0.5)
    cell = tf.contrib.rnn.MultiRNNCell([lstm] * rnn_layers)
    InitialState = cell.zero_state(batch_size, tf.float32)
    InitialState = tf.identity(InitialState, name='initial_state')
    return cell, InitialState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [16]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1, 1), name='embedding')
    embed = tf.nn.embedding_lookup(embedding, input_data, name='embed')
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [17]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32, swap_memory=True)
    FinalState = tf.identity(state, name='final_state')
    return outputs, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


```
# just checking output and state shapes
with tf.Graph().as_default():
    test_rnn_size = 256
    test_rnn_layer_size = 3
    test_cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(test_rnn_size)] * test_rnn_layer_size)

    test_inputs = tf.placeholder(tf.float32, [None, None, test_rnn_size])
    outputs, final_state = build_rnn(test_cell, test_inputs)
    print('cell.state_size:', test_cell.state_size)
    print(type(test_cell.state_size[0]))
    print('cell.output_size:', test_cell.output_size)
    print('final_state shape:', final_state.get_shape())
    print('outputs shape:', outputs.get_shape())

"""
cell.state_size: (LSTMStateTuple(c=256, h=256), LSTMStateTuple(c=256, h=256), LSTMStateTuple(c=256, h=256))
<class 'tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.LSTMStateTuple'>
cell.output_size: 256
final_state shape: (3, 2, ?, 256)
outputs shape: (?, ?, 256)
"""
```

### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [18]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim=300):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim=embed_dim)
    Outputs, FinalState = build_rnn(cell, embed)
    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size, activation_fn=tf.identity)
    
    return Logits, FinalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [19]:
type(int_text)

list

```
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    # int_text is divided into batches with (batch_size, seq_length) in each batch
    num_batches = int(len(int_text)/(batch_size*seq_length))
    batches = []
    #print('num_batches:', num_batches)
    for batches_i in range(num_batches):
        batch_input = []
        batch_target = []
        for ii in range(batch_size):
            start = batches_i * seq_length + ii * num_batches * seq_length
            end = start + seq_length
            batch_input.append(int_text[start:end])
            batch_target.append(int_text[start+1:end+1])
        #print(batch_input)
        #print('batch_input shape:', np.array(batch_input).shape)
        batches.append([batch_input, batch_target])
    return np.array(batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)
```
numpy reshape method is 3x faster

In [20]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    int_text = np.array(int_text)
    # int_text is divided into batches with (batch_size * seq_length) in each batch
    num_batches = int(len(int_text)/(batch_size * seq_length))
    complete_batches_size = num_batches * batch_size * seq_length
    # target is just the next word
    int_target = int_text[1:complete_batches_size + 1]
    # truncate int_text for reshape
    int_text = int_text[:complete_batches_size]
    
    # split into batch_size buckets 
    new_shape = (batch_size, num_batches, seq_length)
    int_text = int_text.reshape(new_shape)
    int_target = int_target.reshape(new_shape)
    
    batches = []
    for ii in range(num_batches):
        # squeeze to reshape from (batch_size, 1, seq_length) to (batch_size, seq_length)
        batch_input = int_text[:, ii:ii+1].squeeze()
        batch_target = int_target[:, ii:ii+1].squeeze()
        
        batches.append([batch_input, batch_target])
    return np.array(batches)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [34]:
# Number of Epochs
num_epochs = 250
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 7

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [36]:
import time
start = time.time()

In [37]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.820
Epoch   0 Batch    7/21   train_loss = 7.949
Epoch   0 Batch   14/21   train_loss = 6.449
Epoch   1 Batch    0/21   train_loss = 6.128
Epoch   1 Batch    7/21   train_loss = 6.099
Epoch   1 Batch   14/21   train_loss = 5.943
Epoch   2 Batch    0/21   train_loss = 5.997
Epoch   2 Batch    7/21   train_loss = 5.871
Epoch   2 Batch   14/21   train_loss = 5.820
Epoch   3 Batch    0/21   train_loss = 5.819
Epoch   3 Batch    7/21   train_loss = 5.683
Epoch   3 Batch   14/21   train_loss = 5.648
Epoch   4 Batch    0/21   train_loss = 5.641
Epoch   4 Batch    7/21   train_loss = 5.511
Epoch   4 Batch   14/21   train_loss = 5.494
Epoch   5 Batch    0/21   train_loss = 5.492
Epoch   5 Batch    7/21   train_loss = 5.361
Epoch   5 Batch   14/21   train_loss = 5.357
Epoch   6 Batch    0/21   train_loss = 5.359
Epoch   6 Batch    7/21   train_loss = 5.227
Epoch   6 Batch   14/21   train_loss = 5.229
Epoch   7 Batch    0/21   train_loss = 5.236
Epoch   7 

In [38]:
print('Training time: {}s'.format(time.time()-start))

Training time: 1120.3344430923462s


In [39]:
batches.shape

(21, 2, 64, 50)

In [40]:
len(int_text)

69101

```
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 200
# Learning Rate
learning_rate = 0.001

ResourceExhaustedError: OOM when allocating tensor with shape[25600,6780]
```

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [43]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [44]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    c = np.random.choice(list(int_to_vocab.keys()), 1, p=probabilities)[0]
    return int_to_vocab[c]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [45]:
gen_length = 400
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(into phone) uh, that's always about my daughter.
moe_szyslak:(shocked) are you nuts? i learned something about myself.
homer_simpson:(sly) are you nuts? i was just drawin' wangs on the numbers.
homer_simpson:(computer voice) yes, i wonder what's the big loser?
lenny_leonard: well, i'm just gonna let anything you've got to lose my favorite meal, the rope will come in some dingy flophouse.
dan_gillick: really?(hopeful) in-ground is in the" flaming homers"?


seymour_skinner: who is that windex?(to moe) she's a state runners up in advance.
lenny_leonard: huh?
lenny_leonard: not true!
barney_gumble: so, i'm just gonna gamble with my daughter's future.
moe_szyslak: nah, come here.
harv: we feel about the new life in fact, chug-monkeys. ooh, ooh, taking advantage of my generous treasure...(third head) it's all relative. that's right, everybody a minute-- this is the big time.
homer_simpson: okay, burger king! whatever! c'mon, moe.
homer_simpson: thanks, moe.(laughs)
moe_szyslak:

num_epochs = 200
```
moe_szyslak: no, the kids you go-- he wrote the parking plan.
moe_szyslak: a big cat before. how'd i do it seem like ya...
homer_simpson: moe, i'm a british pub! i'm a squirrel squashing, you left a friend i'm gonna have.
barney_gumble: hey homer, sorry. get everything while they please/ still then. we want you to play.
c. bring her with a whatchacallit, homer.
homer_simpson: moe, that's funny and ever had same kid.
homer_simpson: barney! ow! thank god, you crazy dames i think so sudden.
homer_simpson: four little bart.(dramatic, omigod, then instantly sober, and i ever without no one ever orders that.
homer_simpson:(murmur understanding assent) listen over.
moe_szyslak: glad you read me more to make ya live.
bill_james: i had a better open with my meals!
larry:(closes phone) yeah!
moe_szyslak: well, we mean bad. we've got a two conference champs must survive a tomorrow, but they wanted it as mr.(dropping character) and love mr. and we belong to the president?
moe_szyslak: i got is time.
carl_carlson:(hurt sigh) ah, boy. think, i gestated for all right. uh, where i have some tanking up.
homer_simpson: nobody meeting.(wistful) all right, alls i was able to school.
artie_ziff: pardon the waterfront oh yeah, looks like the ball.
carl_carlson: grinch.
moe_szyslak: oh, ah, that's all right.
moe_szyslak:(sniffles, then said:(sigh)...(philosophical)"(tapping mic) it'll be eatin' wish-meat every day i even before... seems jacques strap! what the hell kinda here?
anthony_kiedis: you better.
homer_simpson: oh, i think i pop find that how do you won't tell that this place terrible.
homer_simpson: oh, i'm afraid yeah. here, england's springfield isn't against my mom can learn more made myself to win some cotton in the ladies-- and all i should
```

num_epochs = 100
```
moe_szyslak: ooh!(peeved) ah..... and if anybody wants potato chips or anything fancy, even before i can take my vacations someplace hot.
moe_szyslak: oh yeah. can i ever think it's dangerous.
barney_gumble: here you mind ridin' on fire.
moe_szyslak:(struggling noises, then) god, for the day... well, when you do is your money...
lenny_leonard: well, someone slipped me. there's lots of the while i says who should i never sell an opportunity to because we're living in a world. and you gotta wing it a little bit...
homer_simpson:(sobs) what uh...(bitter) unlike me. it all comes.
agent_miller: homer simpson? i never seen you drive your stupid.
homer_simpson: gimme some peanuts.
homer_simpson:(murderously) you and my trenchant my friend!
chief_wiggum: the south of france..... this bashir kid is muslim, and therefore up here.
homer_simpson:(finishing beer) you took youse up. take it as you i'm proposing is a great. but one day finally. very good. um, what you want to know that.
homer_simpson: well, i just whisper it to spread and i'm an alcoholic. oh crap now, the health inspector?
detective_homer_simpson: all right, all right. but homer_simpson: i know things if they're i compliment your purse / but that seems less funny each time i ran out of five. i did have the keys to make my rent.
lenny_leonard: i got the job if you know... this place is the ultimate brain bleacher... can i can't two of that no numeral can be careful.
rev. _hooper: no, i'm starting my car ain't worth nothin'.(moans) but it didn't die... and if anyone i saw you! we have a scotch and water.
homer_simpson:(considers it) hunting, eh?
barney_gumble:(horrified scream) no...
homer_simpson:(warmly) oh, moe. and you
```

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the beginning of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.